In [1]:
import preprocessing.sinonom_pdf_helper as sn
import preprocessing.vietnamese_pdf_helper as vn
import preprocessing.translate_sn2vn
import alignment
import ocr_correction.corrector as crt
import os
import random
import argparse
from random import seed as seed
from laserembeddings import Laser
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

/home/davidle2810/anaconda3/envs/mm_align/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
laser = Laser()

In [3]:
sn_content = [{'page_number': 10, 'content': [{'bbox': [[3220, 4184], [3375, 4184], [3375, 4672], [3220, 4672]], 'content': '百卄六', 'transliteration': 'bách niệm lục '}, {'bbox': [[2740, 963], [2910, 960], [2929, 4876], [2759, 4879]], 'content': '碎德主吞於代尼朱時徽特蓮天堂晦德主吞固氽嵬', 'transliteration': 'tôi đức chúa thôn ở đời này cho thời huy được lên thiên đường hối đức chúa thôn có mấy ngôi '}, {'bbox': [[2509, 1001], [2676, 997], [2698, 5031], [2532, 5034]], 'content': '疎德主吞固巴嵬嵬壹羅吒嵬台羅昆嵬巴羅聖神一晦嵬', 'transliteration': 'thưa đức chúa thôn có ba ngùi ngùi nhất là cha ngôi thay là con ngôi ba là thánh thần nhất hối ngôi '}, {'bbox': [[2283, 963], [2449, 960], [2457, 5023], [2286, 5027]], 'content': '壹羅吒固沛德主吞庄疎沛晦嵬年羅昆固沛德主吞', 'transliteration': 'nhất là cha có phải đức chúa thôn trang thơ phải hối ngôi niên là con có phải đức chúa thôn '}, {'bbox': [[2033, 963], [2203, 960], [2215, 5027], [2048, 5031]], 'content': '庄疎沛晦嵬巴羅聖神固沛德主吞庄疎沛晦裊', 'transliteration': 'chẳng thưa phải hối ngôi ba là thánh thần có phải đức chúa thôn trang thơ phải hối nẻo '}, {'bbox': [[1791, 960], [1961, 960], [1961, 5034], [1791, 5034]], 'content': '丕固沛巴德主坖庄疎庄固爲世嵬拱沒性沒法丕固沒', 'transliteration': 'vậy có phải ba đức chúa 坖 chẳng thưa chẳng có vì thế ngôi cũng một tính một pháp vậy có một '}, {'bbox': [[1557, 956], [1719, 956], [1719, 1814], [1557, 1814]], 'content': '德主吞麻崔', 'transliteration': 'đức chúa thôn mà thôi '}, {'bbox': [[1145, 2120], [1402, 2120], [1402, 3020], [1145, 3020]], 'content': '段次合', 'transliteration': 'đoạn thứ hợp '}, {'bbox': [[827, 5023], [835, 986], [1005, 990], [994, 5027]], 'content': '晦敵巴嵬固嵬芾岍代庄一疎嵬次的崢代濫得固壳固靈', 'transliteration': 'hối địch ba ngôi có ngôi nào khiên đời chẳng nhất thưa ngôi thứ đích tranh đời làm được có xác có linh '}, {'bbox': [[548, 5016], [563, 956], [729, 960], [714, 5019]], 'content': '魂如些丕一晦嵬次台岼代祕壳吧靈魂於兠一疎祕壳吧', 'transliteration': 'hồn như ta vậy nhất hối ngôi thứ thay 岼 đời bí xác và linh hồn ở đâu nhất thơ bí xác và '}]}, {'page_number': 11, 'content': [{'bbox': [[2910, 1908], [3171, 1908], [3171, 3466], [2910, 3466]], 'content': '本玳仍祕昆', 'transliteration': 'bản đồi những bí con '}, {'bbox': [[2543, 2218], [2789, 2218], [2789, 3129], [2543, 3129]], 'content': '段次壹', 'transliteration': 'đoạn thứ nhất '}, {'bbox': [[2264, 956], [2434, 956], [2434, 5053], [2264, 5053]], 'content': '晦德主坴羅等芾疎德主至羅等孕年吞坦閉物共恒僵', 'transliteration': 'hối đức chúa lục là đấng nào thưa đức chúa chí là đứng dựng niên thôn đất bấy vật cùng hằng cương '}, {'bbox': [[2014, 918], [2184, 918], [2184, 4868], [2014, 4868]], 'content': '件該治每事晦固殼德主埀庄疎固沒德主至麻崔', 'transliteration': 'kiện cai trị mọi sự hối có xác đức chúa 埀 chẳng thưa có một đức chúa chí mà thôi '}, {'bbox': [[1765, 5050], [1765, 956], [1939, 956], [1939, 5050]], 'content': '晦德主吞於兠疎德主吞於運吞酃坦於泣每坭晦德', 'transliteration': 'hối đức chúa thôn ở đâu thưa đức chúa thôn ở vận thôn linh đất ở khắp mọi nơi hối đức '}, {'bbox': [[1515, 918], [1689, 914], [1693, 5031], [1523, 5034]], 'content': '主蚕於泣每坭牢低些祜庄筧疎爲德主季羅性聲靈朱', 'transliteration': 'chúa tằm ở khắp mọi nơi sao đây ta hỗ chẳng kiển thưa vì đức chúa quý là tính thanh linh cho '}, {'bbox': [[1270, 922], [1443, 918], [1447, 5050], [1277, 5053]], 'content': '年昆相餂些祜庄賃晦固欺芮麻庄固德主吞庄疎德', 'transliteration': 'niên con tương điềm ta hỗ chẳng nhấm hối có khi nuôi mà chẳng có đức chúa thôn chẳng thưa đức '}, {'bbox': [[1028, 914], [1202, 910], [1213, 5038], [1039, 5042]], 'content': '主岙恒固畧撥無窮庄固包賖麻庄固德主坖晦德主吞', 'transliteration': 'chúa diệu hằng có lược bát vô cùng chẳng có bao xa mà chẳng có đức chúa 坖 hối đức chúa thôn '}, {'bbox': [[778, 918], [948, 914], [960, 5046], [793, 5050]], 'content': '孕年類得些固意芾疎固意朱得些別吧祿奉敬勉共濫', 'transliteration': 'dựng nên loài đắc ta có ý nào thơ có ý cho được ta biết và lộc phụng kính mến cùng làm '}, {'bbox': [[328, 1330], [483, 1330], [483, 2109], [328, 2109]], 'content': '聖教要理', 'transliteration': 'thánh giáo yếu lý '}, {'bbox': [[340, 4154], [487, 4154], [487, 4645], [340, 4645]], 'content': '百卄五', 'transliteration': 'bách nhập ngũ '}]}]
vn_content = [{'page_number': 1, 'content': '— BỒN DẠY NHỮNG TRE CON.!\n\nĐOAN THỨ NHẤT\n\nHỏi. Đức Chúa Trời là Đấng nào? Thưa. Đức Chúa Trời là Đấng dựng nên trời đất muôn vật cùng hãng gìn giữ cai trị mọi sự. Hỏi. Có nhiều Đức Chúa Trời chăng? Thưa. Có một Đức Chúa Trời mà thôi.\nHỏi. Đức Chúa Trời ở đâu? Thưa. Đức Chúa Trời ở trên trời dưới đất ở khắp mọi nơi. Hỏi. Đức Chúa Trời ở khắp mọi nơi sao đây ta xem chăng thấy? Thưa. Vì Đức Chúa Trời là tính thiêng liêng cho nên con mắt thịt ta xem chăng thấy. Hỏi. Có khi nào mà chắng có Đức Chúa Trời chăng? Thưa. Đức Chúa Trời hằng có trước sau vô cùng, chăng có bao giờ mà chăng có Đức Chúa Trời. Hỏi. Đức Chúa Trời dựng nên loài người ta có ý nào? Thưa. Có ý cho người ta biết và thò! phượng kính mến cùng làm.\n\nTHÁNH GIÁO YỂU LÝ - bách chập ngũ (125) \n\n1. Trích Thánh Giáo Yếu Lý, Kẻ Sở, 1933 - bản phiên âm của Nhóm Phiên Dịch Thuật Hán Nôm Công Giáo,  Lm. Vĩnh Sơn Nguyên Hưng chủ biên. 2. Chữ øìn trong mộc bản là chữ nhân + chữ gìn.\n3. Chữ xem trong mộc bản là bộ kỳ + chữ xem. Đúng ra là bộ + chữ xem.\n4. Chữ fhờ trong mộc bản là kỳ (thần) + (tôi, ta). Vì thiếu nên tạm dùng chữ fhờ  bộ (sâu bọ) + (tôi, ta).\n'}, {'page_number': 2, 'content': 'Hỏi. Đức Chúa Trời ở đời này, cho ngày sau được lên thiên đàng. Hỏi. Đức Chúa Trời có mấy ngôi?\nThưa. Đức Chúa Trời có ba ngôi, Ngôi Nhất là Cha, Ngôi Hai là Con, Ngôi Ba là Thánh Thần. Hỏi. Ngôi Nhất là Cha có phải Đức Chúa Trời chăng? Thưa. Phải. Hỏi. Ngôi Hai là Con có phải Đức Chúa Trời chăng? Thưa. Phải. Hỏi. Ngôi Ba là Thánh Thân có phải Đức Chúa Trời chăng? Thưa. Phải. Hỏi. Nếu vậy có phải Ba Đức Chúa Trời chăng? Thưa. Chăng có, vì Ba Ngôi cũng một tính một phép. Vậy có một Đức Chúa Trời mà thôi.\n\nĐOAN THỨ HAI\nHỏi. Trong Ba Ngôi có Ngôi nào ra đời chăng? Thưa. Ngôi Thứ Hai ra đời làm người có xác có linh hồn như ta vậy. Hỏi. Ngôi Thứ Hai ra đời lấy xác và linh hồn ở đâu? Thưa. Lấy xác và \n'}]

In [4]:
print(sn_content)

[{'page_number': 10, 'content': [{'bbox': [[3220, 4184], [3375, 4184], [3375, 4672], [3220, 4672]], 'content': '百卄六', 'transliteration': 'bách niệm lục '}, {'bbox': [[2740, 963], [2910, 960], [2929, 4876], [2759, 4879]], 'content': '碎德主吞於代尼朱時徽特蓮天堂晦德主吞固氽嵬', 'transliteration': 'tôi đức chúa thôn ở đời này cho thời huy được lên thiên đường hối đức chúa thôn có mấy ngôi '}, {'bbox': [[2509, 1001], [2676, 997], [2698, 5031], [2532, 5034]], 'content': '疎德主吞固巴嵬嵬壹羅吒嵬台羅昆嵬巴羅聖神一晦嵬', 'transliteration': 'thưa đức chúa thôn có ba ngùi ngùi nhất là cha ngôi thay là con ngôi ba là thánh thần nhất hối ngôi '}, {'bbox': [[2283, 963], [2449, 960], [2457, 5023], [2286, 5027]], 'content': '壹羅吒固沛德主吞庄疎沛晦嵬年羅昆固沛德主吞', 'transliteration': 'nhất là cha có phải đức chúa thôn trang thơ phải hối ngôi niên là con có phải đức chúa thôn '}, {'bbox': [[2033, 963], [2203, 960], [2215, 5027], [2048, 5031]], 'content': '庄疎沛晦嵬巴羅聖神固沛德主吞庄疎沛晦裊', 'transliteration': 'chẳng thưa phải hối ngôi ba là thánh thần có phải đức chúa thôn

In [5]:
print(vn_content)

[{'page_number': 1, 'content': '— BỒN DẠY NHỮNG TRE CON.!\n\nĐOAN THỨ NHẤT\n\nHỏi. Đức Chúa Trời là Đấng nào? Thưa. Đức Chúa Trời là Đấng dựng nên trời đất muôn vật cùng hãng gìn giữ cai trị mọi sự. Hỏi. Có nhiều Đức Chúa Trời chăng? Thưa. Có một Đức Chúa Trời mà thôi.\nHỏi. Đức Chúa Trời ở đâu? Thưa. Đức Chúa Trời ở trên trời dưới đất ở khắp mọi nơi. Hỏi. Đức Chúa Trời ở khắp mọi nơi sao đây ta xem chăng thấy? Thưa. Vì Đức Chúa Trời là tính thiêng liêng cho nên con mắt thịt ta xem chăng thấy. Hỏi. Có khi nào mà chắng có Đức Chúa Trời chăng? Thưa. Đức Chúa Trời hằng có trước sau vô cùng, chăng có bao giờ mà chăng có Đức Chúa Trời. Hỏi. Đức Chúa Trời dựng nên loài người ta có ý nào? Thưa. Có ý cho người ta biết và thò! phượng kính mến cùng làm.\n\nTHÁNH GIÁO YỂU LÝ - bách chập ngũ (125) \n\n1. Trích Thánh Giáo Yếu Lý, Kẻ Sở, 1933 - bản phiên âm của Nhóm Phiên Dịch Thuật Hán Nôm Công Giáo,  Lm. Vĩnh Sơn Nguyên Hưng chủ biên. 2. Chữ øìn trong mộc bản là chữ nhân + chữ gìn.\n3. Chữ xem tro

In [6]:
sn2vn_content = list()
for page in sn_content:
    sn2vn_content.append({'page_number': page['page_number'], 'content': '\n'.join([line['transliteration'] for line in page['content']])})
    # align paragraphs
paragraph_alignments = alignment.paragraph_alignment(sn2vn_content,vn_content)

In [7]:
print(paragraph_alignments)


[{'source_page_number': 0, 'source_text': 'bách niệm lục \ntôi đức chúa thôn ở đời này cho thời huy được lên thiên đường hối đức chúa thôn có mấy ngôi \nthưa đức chúa thôn có ba ngùi ngùi nhất là cha ngôi thay là con ngôi ba là thánh thần nhất hối ngôi \nnhất là cha có phải đức chúa thôn trang thơ phải hối ngôi niên là con có phải đức chúa thôn \nchẳng thưa phải hối ngôi ba là thánh thần có phải đức chúa thôn trang thơ phải hối nẻo \nvậy có phải ba đức chúa 坖 chẳng thưa chẳng có vì thế ngôi cũng một tính một pháp vậy có một \nđức chúa thôn mà thôi \nđoạn thứ hợp \nhối địch ba ngôi có ngôi nào khiên đời chẳng nhất thưa ngôi thứ đích tranh đời làm được có xác có linh \nhồn như ta vậy nhất hối ngôi thứ thay 岼 đời bí xác và linh hồn ở đâu nhất thơ bí xác và ', 'target_page_number': 1, 'target_text': 'Hỏi. Đức Chúa Trời ở đời này, cho ngày sau được lên thiên đàng. Hỏi. Đức Chúa Trời có mấy ngôi?\nThưa. Đức Chúa Trời có ba ngôi, Ngôi Nhất là Cha, Ngôi Hai là Con, Ngôi Ba là Thánh Thần. Hỏi. 

In [18]:
os.chdir('../')

In [23]:
result = list()
for idx, alignment in enumerate(paragraph_alignments):
    src_id = alignment['source_page_number']
    tgt_id = alignment['target_page_number']
    path_temp_file_sn2vn = os.path.join('data', "test.tr")
    path_temp_file_vn = os.path.join('data', "test.vn")
    with open(path_temp_file_sn2vn, "w", encoding="utf-8") as f:
        f.write(sn2vn_content[src_id]['content'])
    with open(path_temp_file_vn, "w", encoding="utf-8") as f:
        f.write(vn_content[tgt_id]['content'])
    os.chdir('./sentence_alignments')
    script = './align.sh' 
    print(f"Running: {script}")
    os.system(script)
    os.chdir('../')

Running: ./align.sh


../../LASER/tasks/embed/embed.sh: 64: [[: not found
Traceback (most recent call last):
  File "/mnt/d/LASER/source/embed.py", line 37, in <module>
    from laser_encoders.models import SentenceEncoder
  File "/mnt/d/LASER/laser_encoders/__init__.py", line 16, in <module>
    from laser_encoders.models import LaserEncoderPipeline, initialize_encoder
  File "/mnt/d/LASER/laser_encoders/models.py", line 27, in <module>
    from fairseq.data.dictionary import Dictionary
ModuleNotFoundError: No module named 'fairseq'
../../LASER/tasks/embed/embed.sh: 64: [[: not found
Traceback (most recent call last):
  File "/mnt/d/LASER/source/embed.py", line 37, in <module>
    from laser_encoders.models import SentenceEncoder
  File "/mnt/d/LASER/laser_encoders/__init__.py", line 16, in <module>
    from laser_encoders.models import LaserEncoderPipeline, initialize_encoder
  File "/mnt/d/LASER/laser_encoders/models.py", line 27, in <module>
    from fairseq.data.dictionary import Dictionary
ModuleNotFo

Running: ./align.sh


../../LASER/tasks/embed/embed.sh: 64: [[: not found
Traceback (most recent call last):
  File "/mnt/d/LASER/source/embed.py", line 37, in <module>
    from laser_encoders.models import SentenceEncoder
  File "/mnt/d/LASER/laser_encoders/__init__.py", line 16, in <module>
    from laser_encoders.models import LaserEncoderPipeline, initialize_encoder
  File "/mnt/d/LASER/laser_encoders/models.py", line 27, in <module>
    from fairseq.data.dictionary import Dictionary
ModuleNotFoundError: No module named 'fairseq'
../../LASER/tasks/embed/embed.sh: 64: [[: not found
Traceback (most recent call last):
  File "/mnt/d/LASER/source/embed.py", line 37, in <module>
    from laser_encoders.models import SentenceEncoder
  File "/mnt/d/LASER/laser_encoders/__init__.py", line 16, in <module>
    from laser_encoders.models import LaserEncoderPipeline, initialize_encoder
  File "/mnt/d/LASER/laser_encoders/models.py", line 27, in <module>
    from fairseq.data.dictionary import Dictionary
ModuleNotFo